# Clone the repo
Get the repo and submodules

In [1]:
!git clone https://github.com/awskids/logo-model-spike.git

Cloning into 'logo-model-spike'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 66 (delta 5), reused 12 (delta 5), pack-reused 48
Unpacking objects: 100% (66/66), 78.73 MiB | 13.89 MiB/s, done.


Download submodules.

In [2]:
%cd logo-model-spike/
%cd yolov7-logo-main/
!git submodule update --init


/content/logo-model-spike
/content/logo-model-spike/yolov7-logo-main


In [5]:
!pwd
%cd src
!git clone https://github.com/WongKinYiu/yolov7.git
%cd ..

/content/logo-model-spike/yolov7-logo-main
/content/logo-model-spike/yolov7-logo-main/src
Cloning into 'yolov7'...
remote: Enumerating objects: 1130, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1130 (delta 0), reused 1 (delta 0), pack-reused 1127
Receiving objects: 100% (1130/1130), 69.94 MiB | 44.32 MiB/s, done.
Resolving deltas: 100% (519/519), done.
/content/logo-model-spike/yolov7-logo-main


Install the necessary packages.

In [6]:
!pip install -r src/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.2 MB/s eta 0:00:00


# Download the dataset
Run the **getFlickr.sh** file to download the dataset.

In [7]:
!sh data/getFlickr.sh

--2023-03-11 22:58:09--  http://image.ntua.gr/iva/datasets/flickr_logos/flickr_logos_27_dataset.tar.gz
Resolving image.ntua.gr (image.ntua.gr)... 147.102.11.1
Connecting to image.ntua.gr (image.ntua.gr)|147.102.11.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101827904 (97M) [application/x-gzip]
Saving to: ‘./data/flickr_logos_27_dataset.tar.gz’

flickr_logos_27_dat 100%[===================>]  97.11M  5.79MB/s    in 25s     

2023-03-11 22:58:34 (3.85 MB/s) - ‘./data/flickr_logos_27_dataset.tar.gz’ saved [101827904/101827904]

flickr_logos_27_dataset/
flickr_logos_27_dataset/flickr_logos_27_dataset_distractor_set_urls.txt
flickr_logos_27_dataset/flickr_logos_27_dataset_training_set_annotation.txt
flickr_logos_27_dataset/flickr_logos_27_dataset_query_set_annotation.txt
flickr_logos_27_dataset/flickr_logos_27_dataset_images.tar.gz
flickr_logos_27_dataset_images/
flickr_logos_27_dataset_images/2176584286.jpg
flickr_logos_27_dataset_images/3156402215.jpg
flickr

# Prepare data
We nee to convert data to YOLO format.Now that we have our dataset, we need to convert the annotations into the format expected by YOLOv7. YOLOv7 expects data to be organized in a specific way, otherwise it is unable to parse through the directories.

In [8]:
!python src/convert_annotations.py --dataset flickr27

100% 1079/1079 [00:03<00:00, 329.58it/s]


To see if the conversion is correct, run.

In [9]:
!python src/convert_annotations.py --dataset flickr27 --plot

100% 1079/1079 [00:03<00:00, 334.36it/s]
Traceback (most recent call last):
  File "/content/logo-model-spike/yolov7-logo-main/src/convert_annotations.py", line 246, in <module>
    main()
  File "/content/logo-model-spike/yolov7-logo-main/src/convert_annotations.py", line 242, in main
    plot_bounding_box(image, annotation_list)
TypeError: plot_bounding_box() missing 1 required positional argument: 'class_id_to_name_mapping'


Then, split data into sets.Next, we need to partition the dataset into train, validation, and test sets. These will contain 80%, 10%, and 10% of the data, respectively.

In [10]:
!python src/prepare_data.py --dataset flickr27

# Start training

In [11]:
!python src/yolov7/train.py --img-size 640 --cfg src/cfg/training/yolov7.yaml --hyp data/hyp.scratch.yaml --batch 2 --epoch 300 --data data/logo_data_flickr.yaml --weights src/yolov7_training.pt --workers 2 --name yolo_logo_det --device 0

YOLOR 🚀 v0.1-122-g3b41c2c torch 1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40513.5625MB)

Namespace(weights='src/yolov7_training.pt', cfg='src/cfg/training/yolov7.yaml', data='data/logo_data_flickr.yaml', hyp='data/hyp.scratch.yaml', epochs=300, batch_size=2, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=2, project='runs/train', entity=None, name='yolo_logo_det', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/yolo_logo_det', total_batch_size=2)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2023-03-11 22:59:13.858666: I tensorflow/core/platform/cpu_feature